In [ ]:

from objectnat import get_intermodal_graph
import geopandas as gpd

G_intermodal = get_intermodal_graph(osm_id=1114252)


In [ ]:
from objectnat import get_isochrone_zone_coverage
services = gpd.read_parquet('examples_data/services.parquet').to_crs(32636).iloc[:5]
isochrones, stops, routes  = get_isochrone_zone_coverage(
    services=services,
    weight_type="time_min",
    weight_value=10,
    city_graph=G_intermodal,
)

In [ ]:
m = isochrones.explore(column='id',categorical=True,tiles='CartoDB positron')
stops.explore(m=m)
routes.explore(m=m, column='type')